In [21]:
import textstat

#text = "Graphics library to create an embedded GUI with easy-to-use graphical elements, beautiful visual effects and low memory footprint. It offers anti-aliasing, opacity, and animations using only one frame buffer. "
#text = "Alternative firmware for ESP8266 and ESP32 based devices with easy configuration using webUI, OTA updates, automation using timers or rules, expandability and entirely local control over MQTT, HTTP, Serial or KNX. Full documentation at"
text = "my exp for CVE-2020-27194, tested on linux kernel 5.8.14."
#text = "🎬 IMDb: Link 'em all!  -  Adds all kinds of links to IMDb, customizable!"
#text = "A sexy achievement file parser with real-time notification, automatic screenshot and playtime tracking. View every achievements earned on your PC whether it's coming from Steam, a Steam emulator, and more."
#text = "Complete-Python-3-Bootcamp"

In [22]:
print(textstat.flesch_reading_ease(text))
print(textstat.dale_chall_readability_score(text))
print(textstat.gulpease_index(text))
print(textstat.osman(text))

87.72
21.63
67.9
68.06
